In [ ]:
from dotenv import load_dotenv
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

load_dotenv()

<img src="../images/openai-example.png" width="850" height="500">

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI

In [ ]:
hub.pull("hwchase17/openai-tools-agent").invoke({"input": "What is the most populated city in the world?", "agent_scratchpad":[]}).to_string()

<h1>Define the tools</h1>
<img src="../images/openai-example-tools.png" width="850" height="500">

In [ ]:
from typing import Dict
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_community.tools import DuckDuckGoSearchRun


# Define the tools
@tool
def characters_count(word: str) -> Dict[str, int]:
    """This tool count the number of characters in a word and returns the number. use this tool only if the intent of the user is to count characters"""
    return { "response": len(word) }

tools = [characters_count, DuckDuckGoSearchRun()]
[convert_to_openai_tool(tool) for tool in tools]

<img src="../images/openai-routing-strategy.png" width="850" height="500">

In [ ]:

prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
# llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])
# 
# agent = (
#     RunnablePassthrough.assign(
#         agent_scratchpad=lambda x: format_to_openai_tool_messages(
#             x["intermediate_steps"]
#         )
#     )
#     | prompt
#     | llm_with_tools
#     | OpenAIToolsAgentOutputParser()
#     )

agent = create_openai_tools_agent(llm, tools, prompt)


<img src="../images/openao-tool-exec.png" width="850" height="500">

In [ ]:
print(agent.invoke({"input": "What is the most populated city in the world?","intermediate_steps":""}))

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "how many chars in the middle name of will smith?"})

<a href="https://smith.langchain.com/o/a25e8a2d-5312-4d0d-9f0c-c9311a37078b/projects/p/4afa5f3c-6ac1-44a2-a8a5-4e917b7a8bc5?columnVisibilityModel=%7B%22outputs%22%3Afalse%2C%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%7D&timeModel=%7B%22duration%22%3A%227d%22%7D">Langsmith</a>

<img src="../images/prompt-eng-in-langsmith.png" width="850" height="500">

In [ ]:
agent_executor.invoke({"input": "how many chars in the middle name of will smith? don't count on your previous infomration search the web instead"})
